In [1]:
import requests

channel_id = #
api_key = #
url_video_base = "https://www.youtube.com/watch?v="

In [2]:
url_channel = f"https://www.googleapis.com/youtube/v3/channels"

channel_response = requests.get(url_channel, params={"id": channel_id, "key": api_key, "part": "snippet,contentDetails"})
channel_response_body = channel_response.json()
playlist_id = channel_response_body['items'][0]['contentDetails']['relatedPlaylists']['uploads']

In [3]:
url_playlist = f"https://www.googleapis.com/youtube/v3/playlistItems"

playlist_response = requests.get(url_playlist, params={
    "playlistId": playlist_id, 
    "key": api_key, 
    "part": "snippet,id,contentDetails", 
    "maxResults": 50})

In [4]:
video_url_list = {}

In [5]:
while True:
    playlist_response_body = playlist_response.json()

    for video_item in playlist_response_body['items']:
        video_title = video_item['snippet']['title']
        video_id = video_item['snippet']['resourceId']['videoId']
        url_video = url_video_base+video_id
        video_url_list[video_title] = url_video
        
    if 'nextPageToken' in playlist_response_body:
        playlist_response = requests.get(url_playlist, params={
            "playlistId": playlist_id, 
            "key": api_key, 
            "part": "snippet,id,contentDetails", 
            "maxResults": 50,
            "pageToken": playlist_response_body['nextPageToken']})
    else:
        break

In [23]:
import pandas as pd
import numpy as np

In [20]:
video_url_list = list(video_url_list.values())
video_len = len(video_url_list)

In [30]:
datas = np.array([range(600, 600+video_len), video_url_list, [1]*video_len, [0]*video_len, [0]*video_len, [0]*video_len, [0]*video_len])
datas = datas.T

In [33]:
df = pd.DataFrame(datas, columns=['id', 'url', 'train/val', 'minY', 'maxY', 'minX', 'maxX'])

In [35]:
df.to_csv("./data/PianoYT/yt_kassia.csv", index=False, header=False)